# Install and import libraries

In [1]:
import sys
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import glob
from sklearn.model_selection import train_test_split

import math
import gc

np.random.seed(10)
tf.random.set_seed(10)

# Verify GPU device

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Prepare simulated client data

In [3]:
"""
@ The method is to divide the whole data set into 'K' different clients / sets based on the labels.
@ This is to define independent data division

If K = 1, then all the labels belong to `1` set.
If K = 5, then 100/20=5 sets are created with 20 items in each set.
"""
def filter_client_data(images, labels, K):
    filtered_images = []
    filtered_labels = []
    selector = []

    NC = 10
    Step = int(NC/K)
    for i in range(0, NC, Step):
        selector.append(
            np.isin(labels, np.array([j for j in range(i, i+Step)]))
        )


    """
    # MNIST data
    for s in selector:
        filtered_images.append(
            images[s]
        )
        filtered_labels.append(
            labels[s]
        )
    """


    # CIFAR data
    for s in selector:
        filtered_images.append(
            images[s.reshape(images.shape[0])]
        )
        filtered_labels.append(
            labels[s.reshape(images.shape[0])]
        )



    return filtered_images, filtered_labels

In [4]:
"""
@ This is to define mixed data division

If K = 1, then all the labels belong to `1` set.
If K = 5, then 100/20=5 sets are created with 20 items in each set.
"""
def filter_homogenous_client_data(images, labels, K):
    filtered_images = []
    filtered_labels = []
    selector = []

    for i in range(K):
        selector.append(
            np.random.choice(len(labels), int(len(labels)/K), replace=False)
        )


    for s in selector:
        filtered_images.append(
            images[s]
        )
        filtered_labels.append(
            labels[s]
        )

    return filtered_images, filtered_labels

# Select test sample for core algorithm testing

In [5]:
def pick_random_sample_test_data(images, labels, split):
    x = []
    y = []
    clients = len(images)
    for i in range(clients):
        indices = np.random.randint( 0, images[i].shape[0], size=int(split*images[i].shape[0]) )
        x.append(
            np.take(images[i], indices, axis=0)
        )
        y.append(
            np.take(labels[i], indices, axis=0)
        )
    return x, y

# Model architecture initialization

In [8]:
def get_init_model():
    # (*) CIFAR
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(2048, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(100, activation="softmax")
    ])

    """
    # (*) MNIST
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation="softmax")
    ])
    """


    return model

# Machine Learning - Utilities

In [9]:
# Model setting weight wrapper
def set_weight_model(model, average_weights):
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    model.set_weights(average_weights)
    return model


In [10]:
# Model fitting weight wrapper
def fit_model(model, x, y):
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    model.fit(x, y, epochs=30, batch_size=512)
    return model

# Federated Learning - utilities

In [11]:
# FL scale model weights based on `weights` or priorities
def fl_scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

In [12]:
# FL summation of scaled weights
def fl_sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    # get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
    return avg_grad


# FedAvg

In [13]:
def fed_avg(models, scaling_factor):
    scaled_local_weight_list = list()
    for m in range(len(models)):
        scaled_weights = fl_scale_model_weights(models[m].get_weights(), scaling_factor[m])
        scaled_local_weight_list.append(scaled_weights)
    fed_average_weights = fl_sum_scaled_weights(scaled_local_weight_list)
    return fed_average_weights

# Local-FedGT

In [14]:
def gt_fed_avg(models, x, y, theta):
    clients = len(models)
    # clustering_x, game_x, clustering_y, game_y = [None for i in range(clients)], [None for i in range(clients)], [None for i in range(clients)], [None for i in range(clients)]
    # for i in range(clients):
    #     clustering_x[i], game_x[i], clustering_y[i], game_y[i] = train_test_split(x[i], y[i], test_size=0.5, random_state=45)
    gt_models = []
    for i in range(clients):
        accuracy = []
        selected_models = []
        self_model = None
        for j in range(clients):
            # select self model separately
            if i == j:
                self_model = models[j]
            _, acc = models[j].evaluate(x[i], y[i], batch_size=512)
            print(f"outer client={i} inner client={j} acc={acc}")
            # acc = models[j].evaluate(clustering_x[i], clustering_y[i], batch_size=512)[1]
            # Clustering selection based on client model's accuracy on
            # self data
            if acc >= theta:
                accuracy.append(
                    acc
                )
                selected_models.append(
                    models[j]
                )

        # Game theory to select Fed Avg weights :
        # Split the test set in two parts and then use 1 set to get the clustering
        # second set to calculate accuracy for weights in fed avg
        _n_ = len(selected_models)
        _sum_ = sum(accuracy)
        # weights are equal for all the clients
        scaling_factor_strategy = [float(1)/float(_n_) for k in range(_n_)]
        print(f"SS GT {scaling_factor_strategy} _n_ {_n_}")
        gt_model_client = set_weight_model(get_init_model(), fed_avg(selected_models, scaling_factor_strategy))

        # game between self_model and aggregated model to improve personalization
        self_priority = 0.0
        aggregated_priority = 1.0
        combined_model = set_weight_model(get_init_model(), fed_avg([self_model, gt_model_client], [self_priority, aggregated_priority]))
        _, combined_acc = combined_model.evaluate(x[i], y[i], batch_size=512)
        print(f"game between self_model and aggregated model init acc = {combined_acc}")
        epochs = 10
        alpha = 0.1
        min_diff_for_next_step = 0.05
        while epochs > 0:
            epochs -= 1
            print(f"game between self_model and aggregated model self_priority = {self_priority} aggregated_priority = {aggregated_priority}")
            model_1 = set_weight_model(get_init_model(), fed_avg([self_model, gt_model_client], [self_priority+alpha, aggregated_priority-alpha]))
            _, acc_1 = model_1.evaluate(x[i], y[i], batch_size=512)
            # model_2 = set_weight_model(get_init_model(), fed_avg([self_model, gt_model_client], [self_priority-alpha, aggregated_priority+alpha]))
            # _, acc_2 = model_2.evaluate(x[i], y[i], batch_size=512)
            print(f"game between self_model and aggregated model acc_1 = {acc_1} at epoch = {epochs}")
            # print(f"game between self_model and aggregated model acc_1 = {acc_2} at epoch = {epochs}")
            # print(f"acc diff = {abs(acc_2 - acc_1)}")
            print(f"acc diff = {abs(combined_acc - acc_1)}")
            if abs(combined_acc - acc_1) > min_diff_for_next_step:
                print("Inside the priority manipulation step")
                if acc_1 > combined_acc:
                    self_priority += alpha
                    aggregated_priority -= alpha
                # else:
                #     self_priority -= alpha
                #     aggregated_priority += alpha
                print(f"game between self_model and aggregated model self_priority = {self_priority} aggregated_priority = {aggregated_priority}")
            else:
                break

        combined_model = set_weight_model(get_init_model(), fed_avg([self_model, gt_model_client], [self_priority, aggregated_priority]))

        gt_models.append(
            combined_model
        )
    return gt_models

In [15]:
def gt_fed_avg_old(models, x, y, theta):
    gt_models = []
    clients = len(models)
    for i in range(clients):
        accuracy = []
        selected_models = []
        for j in range(clients):
            _, acc = models[j].evaluate(x[i], y[i], batch_size=512)
            # Clustering selection based on client model's accuracy on
            # self data
            if acc >= theta:
                accuracy.append(
                    acc
                )
                selected_models.append(
                    models[j]
                )

        # Game theory to select Fed Avg weights
        _n_ = len(selected_models)
        scaling_factor_strategy = [float(k)/float(_n_) for k in accuracy]
        print(f"SS GT {scaling_factor_strategy} _n_ {_n_}")
        gt_models.append(
            set_weight_model(get_init_model(), fed_avg(selected_models, scaling_factor_strategy))
        )
    return gt_models

# Experiment

In [16]:
df_data = {
    "run": [],
    "client": [],
    "fedavg_acc": [],
    "gt_fedavg_acc": []
}

In [20]:
def main(run, K, gt_test_data_split, theta, homogenous=False):
    (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar100.load_data()
    train_images = train_images / 255.0
    test_images = test_images / 255.0
    print(f"Data downloaded for train:{train_images.shape} and test:{test_images.shape}")

    if not homogenous:
        train_x, train_y = filter_client_data(train_images, train_labels, K)
        print(f"Train data filtered as localized client data")

        test_x, test_y = filter_client_data(test_images, test_labels, K)
    else:
        train_x, train_y = filter_homogenous_client_data(train_images, train_labels, K)
        print(f"Train data filtered as localized client data")

        test_x, test_y = filter_homogenous_client_data(test_images, test_labels, K)

    game_x, game_y = pick_random_sample_test_data(test_x, test_y, gt_test_data_split)
    print(f"Sampling from test data completed for game rounds in game theory federated average")

    local_models = []

    fed_avg_scaling_factor = [float(1/K) for i in range(K)]

    clients = len(train_x)

    for i in range(clients):
        local_models.append(
            fit_model(get_init_model(), train_x[i], train_y[i])
        )
        print(f"Local model build completed for client={i+1}")

    fed_avg_model = set_weight_model(get_init_model(), fed_avg(local_models, fed_avg_scaling_factor))
    print(f"Federated average aggregation completed")

    gt_fed_avg_models = gt_fed_avg(local_models, game_x, game_y, theta)
    print(f"Game theory based federated average aggregation completed")

    for j in range(clients):
        df_data["run"].append(run)

        df_data["client"].append(j+1)

        _, fedavg_acc = fed_avg_model.evaluate(test_x[j], test_y[j], batch_size=512)
        df_data["fedavg_acc"].append(fedavg_acc)

        _, gt_fedavg_acc = gt_fed_avg_models[j].evaluate(test_x[j], test_y[j], batch_size=512)
        df_data["gt_fedavg_acc"].append(gt_fedavg_acc)

        print(f"Debug data {fed_avg_model.evaluate(test_x[j], test_y[j], batch_size=512)}")

        print(f"Evaluation completed for client={j+1}")

In [21]:
def driver(K_param, gts_param, theta_param, homogenous_param):
    runs = 1
    K = int(K_param)
    gts = float(gts_param)
    theta = float(theta_param)
    homogenous = int(homogenous_param)
    homogenity = {
        0: False,
        1: True
    }
    for run_id in range(1,runs+1):
        main(run_id, K, gts, theta, homogenous=homogenity[homogenous])

    df = pd.DataFrame(df_data)
    agg_df = df.groupby('client', as_index=False).mean()[['fedavg_acc', 'gt_fedavg_acc']]
    agg_df  = agg_df.astype(float)
    agg_df.to_csv(f'K={K}_homogenity={str(homogenity[homogenous])}_theta={theta}.csv', index=False)

In [ ]:
# driver(5, 0.5, 0.01, 0)
# driver(10, 0.5, 0.01, 0)
driver(10, 0.5, 0.01, 1)
# K = number of clients,
# gts_param = game theory param for test data split
# theta_param = threshold for GT
# homogenous_param = 0: false, 1: true

# Analysis

In [ ]:
datasets = ['fmnist', 'cifar10', 'cifar100']

base_path = 'results'

for d in datasets:
    files = glob.glob(os.path.join(base_path, d, '*.csv'))
    print(f"Dataset = {d}")
    for f in files:
        df = pd.read_csv(f)
        _K, _h = f.split('_')[0], f.split('_')[1]
        _K = _K.split('=')[1]
        _h = _h.split('=')[1]
        if _K == '5':
            heterogenity = 'EXTREME'
        else:
            if _h == "True":
                heterogenity = 'HOMOGENOUS'
            else:
                heterogenity = 'SEVERE'
        print(f"Heterogenity = {heterogenity} | Average accuracy = {df['gt_fedavg_acc'].mean()}")
    print("")